### 1. Data Extraction

In [20]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

import statistics
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
import math

In [21]:
# Use of Bearer Token

os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAOY3eQEAAAAA%2FK0FPI1vVnopel6JTVkr3Xub%2BlE%3Dpbokl1iPkF9foae0YEnAfsmUBt2jsF0DMss8nvElve8Z8dhoVL'
# os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAIu3dQEAAAAAv%2Bz8dWQYjA0ZOWHJlcu%2FFIUiFjA%3Dn54h9wPaU6iRsCwWKAlanfprENYSf98Rbups3bltouPpdDULUf'

In [22]:
# Auth function to retrieve the bearer token from the environment

def auth():
    return os.getenv('TOKEN')

In [23]:
# Creating headers

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [24]:
# Creating URLs

def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [25]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [26]:
def append_to_csv_updated(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # 9. Possibly sensitive
        reply_settings = tweet['reply_settings']
        
        # 10. Conversation id 
        conversation_id = tweet['conversation_id']
        
        # 11. Referenced tweets type
        if ('referenced_tweets' in tweet):
            referenced_tweets_type = tweet['referenced_tweets']
        else:
            referenced_tweets_type = ""
        
        # 12. Referenced tweets id
        if ('referenced_tweets' in tweet):
            referenced_tweets_id = tweet['referenced_tweets']
        else:
            referenced_tweets_id = ""
        
        # 13. Reply to
        if ('in_reply_to_user_id' in tweet):
            in_reply_to_user_id = tweet['in_reply_to_user_id']
        else:
            in_reply_to_user_id = ""
        
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count,\
               source, text, reply_settings, conversation_id, referenced_tweets_type, referenced_tweets_id, \
               in_reply_to_user_id]
        
       
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [27]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
now = datetime.now()
current_datetime = now.strftime("%d%m_%H%M%S")
filename = 'data' + current_datetime + '.csv'
filename

'data0208_123009.csv'

In [28]:
# Modifying the query based on the parameters
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "erectile dysfunction lang:en"

start_list = []
end_list = []

date = datetime(2017, 1, 1)
start_date = date


for i in range(53):
    start_date_formatted = start_date.isoformat() + '.000Z'
    start_list.append(start_date_formatted)

    end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
    end_date_formatted = end_date.isoformat() + '.000Z'
    end_list.append(end_date_formatted)

    start_date += relativedelta(months=1)


max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open(filename, "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', \
                    'reply_count','retweet_count','source','tweet','reply_settings', 'conversation_id', \
                    'referenced_tweets_type', 'referenced_tweets_id', 'in_reply_to_user_id'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 100 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv_updated(json_response, filename)
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv_updated(json_response, filename)
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  1jzu9lk96gu5npw3j426xzdxdo2y5ycm7k4x0ej3a7p9
Start Date:  2017-01-01T00:00:00.000Z
# of Tweets added from this response:  490
Total # of Tweets added:  490
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  1jzu9lk96gu5npw3j71yw5bazyho8uxq3qs0yktjfcsd
Start Date:  2017-02-01T00:00:00.000Z
# of Tweets added from this response:  488
Total # of Tweets added:  978
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  1jzu9lk96gu5npw3ja2kwxtiz7qu4kvt2ra07kxqovb1
Start Date:  2017-03-01T00:00:00.000Z
# of Tweets added from this response:  489
Total # of Tweets added:  1467
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  1jzu9lk96gu5npw3jd2da20ogafd2kpfloh22196ifel
Start Date:  2017-04-01T00:00:00.000Z
# of Tweets added from this response:  471
Total # of Tweets added:  1938
------------------

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fnm91icm2j7gudg6vph4n42wt31qbh
Start Date:  2019-10-01T00:00:00.000Z
# of Tweets added from this response:  446
Total # of Tweets added:  16346
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6ve98rqrgp6lv5yb4s1k1vkvhv25
Start Date:  2019-11-01T00:00:00.000Z
# of Tweets added from this response:  480
Total # of Tweets added:  16826
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yegcipltm3zzv9aqlolu95izypp
Start Date:  2019-12-01T00:00:00.000Z
# of Tweets added from this response:  477
Total # of Tweets added:  17303
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71enkjfd6wuys17ts5qvvc7ckm7x
Start Date:  2020-01-01T00:00:00.000Z
# of Tweets added from this response:  439
Total # of Tweets added:  17742
--------

### 2. Data Wrangling
### <ensp> 2.a. Cleaning
#### 2.a.i Handling duplicate tweets:

In [29]:
filename = 'data0108_164439.csv'
df_ed = pd.read_csv(filename)

FileNotFoundError: [Errno 2] No such file or directory: 'data0108_164439.csv'

In [ ]:
df_ed.head()

In [ ]:
df = df_ed[df_ed.referenced_tweets_type.isna()]

In [ ]:
print(f'Count of tweets for ED: {len(df_ed)}')

In [ ]:
df.head()

In [ ]:
print(f'Count of tweets for ED after removing the duplicates: {len(df)}')

#### 2.a.ii Hyperlinks:

In [ ]:
def presence(x,text):
    if text in x:
        return 1
    else:
        return 0

In [ ]:
df['hyperlink_tag']=df['tweet'].apply(lambda x: presence(x,'http'))

In [ ]:
df['hyperlink_tag'].value_counts()

In [80]:
import re

In [30]:
df['Tweet_exc_link']=df['tweet'].apply(lambda x: re.sub(r" http\S+|www.\S+", "",x))
df['Tweet_inc_link']=df['tweet'].apply(lambda x: re.findall("(http\S+|www.\S+)",x))
df['Tweet_inc_link']=df['Tweet_inc_link'].apply(lambda x: str(x).replace("[","").replace("]","").strip())

NameError: name 'df' is not defined

In [82]:
df.head()

,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,reply_settings,conversation_id,referenced_tweets_type,referenced_tweets_id,in_reply_to_user_id,hyperlink_tag,Tweet_exc_link,Tweet_inc_link
0,72687927,2017-01-30 23:59:54+00:00,,826218412087058432,en,0,0,0,0,Twitter Web Client,Erectile Dysfunction Download - https://t.co/2...,everyone,826218412087058432,NaN,NaN,NaN,1,Erectile Dysfunction Download - #iTunes #hypno...,'https://t.co/2DOzGXrbVl'
1,810231726396080128,2017-01-30 23:50:19+00:00,,826216000412647424,en,0,0,0,0,WordPress.com,Could acupuncture cure erectile dysfunction? h...,everyone,826216000412647424,NaN,NaN,NaN,1,Could acupuncture cure erectile dysfunction?,"'https://t.co/B0HqZeMgdy', 'https://t.co/mwnHs..."
3,2152440722,2017-01-30 23:45:14+00:00,,826214720780722176,en,1,0,0,0,dlvr.it,Could acupuncture cure erectile dysfunction? h...,everyone,826214720780722176,NaN,NaN,NaN,1,Could acupuncture cure erectile dysfunction?,"'https://t.co/C0XlVKmnGB', 'https://t.co/4I6fh..."
4,2891120914,2017-01-30 23:45:11+00:00,,826214708512296960,en,1,0,0,0,dlvr.it,Could acupuncture cure erectile dysfunction? h...,everyone,826214708512296960,NaN,NaN,NaN,1,Could acupuncture cure erectile dysfunction?,"'https://t.co/q3RiaKo4HE', 'https://t.co/L2lIN..."
5,1956941058,2017-01-30 23:44:02+00:00,,826214419982143488,en,1,0,0,0,IFTTT,Could acupuncture cure erectile dysfunction?\n...,everyone,826214419982143488,NaN,NaN,NaN,1,Could acupuncture cure erectile dysfunction?\n...,"'https://t.co/EG6mffU86k', 'https://t.co/UYAwz..."


#### 2.a.iii Hashtag removal:

In [83]:
df['Tweet_exc_hash']=df['Tweet_exc_link'].apply(lambda x: x.replace('#','').strip())
df['Tweet_hashtag_inc']=df['Tweet_exc_link'].apply(lambda x: re.findall("#(\S+)",x))
df['Tweet_hashtag_inc']=df['Tweet_hashtag_inc'].apply(lambda x: " ".join(x))

C:\Users\Nupur\AppData\Local\Temp/ipykernel_17280/1925447993.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweet_exc_hash']=df['Tweet_exc_link'].apply(lambda x: x.replace('#','').strip())
C:\Users\Nupur\AppData\Local\Temp/ipykernel_17280/1925447993.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweet_hashtag_inc']=df['Tweet_exc_link'].apply(lambda x: re.findall("#(\S+)",x))
C:\Users\Nupur\AppData\Local\Temp/ipykernel_17280/1925447993.py:3: SettingWithCopyWarning: 
A value is trying to be 

#### 2.a.iv Removal of \n, \t, \&gt;

In [84]:
df['tweet_formatted'] = df['Tweet_exc_hash'].apply(lambda x: x.replace('&gt;|\n|\t', ''))

C:\Users\Nupur\AppData\Local\Temp/ipykernel_17280/1711057798.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_formatted'] = df['Tweet_exc_hash'].apply(lambda x: x.replace('&gt;|\n|\t', ''))


In [85]:
# pip install emoji

In [86]:
import emoji

In [87]:
df['tweet_formatted'] = df['tweet_formatted'].apply(lambda x: emoji.demojize(x))

C:\Users\Nupur\AppData\Local\Temp/ipykernel_17280/3760804546.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_formatted'] = df['tweet_formatted'].apply(lambda x: emoji.demojize(x))


In [88]:
df.head()

,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,...,conversation_id,referenced_tweets_type,referenced_tweets_id,in_reply_to_user_id,hyperlink_tag,Tweet_exc_link,Tweet_inc_link,Tweet_exc_hash,Tweet_hashtag_inc,tweet_formatted
0,72687927,2017-01-30 23:59:54+00:00,,826218412087058432,en,0,0,0,0,Twitter Web Client,...,826218412087058432,NaN,NaN,NaN,1,Erectile Dysfunction Download - #iTunes #hypno...,'https://t.co/2DOzGXrbVl',Erectile Dysfunction Download - iTunes hypnoth...,iTunes hypnotherapy,Erectile Dysfunction Download - iTunes hypnoth...
1,810231726396080128,2017-01-30 23:50:19+00:00,,826216000412647424,en,0,0,0,0,WordPress.com,...,826216000412647424,NaN,NaN,NaN,1,Could acupuncture cure erectile dysfunction?,"'https://t.co/B0HqZeMgdy', 'https://t.co/mwnHs...",Could acupuncture cure erectile dysfunction?,,Could acupuncture cure erectile dysfunction?
3,2152440722,2017-01-30 23:45:14+00:00,,826214720780722176,en,1,0,0,0,dlvr.it,...,826214720780722176,NaN,NaN,NaN,1,Could acupuncture cure erectile dysfunction?,"'https://t.co/C0XlVKmnGB', 'https://t.co/4I6fh...",Could acupuncture cure erectile dysfunction?,,Could acupuncture cure erectile dysfunction?
4,2891120914,2017-01-30 23:45:11+00:00,,826214708512296960,en,1,0,0,0,dlvr.it,...,826214708512296960,NaN,NaN,NaN,1,Could acupuncture cure erectile dysfunction?,"'https://t.co/q3RiaKo4HE', 'https://t.co/L2lIN...",Could acupuncture cure erectile dysfunction?,,Could acupuncture cure erectile dysfunction?
5,1956941058,2017-01-30 23:44:02+00:00,,826214419982143488,en,1,0,0,0,IFTTT,...,826214419982143488,NaN,NaN,NaN,1,Could acupuncture cure erectile dysfunction?\n...,"'https://t.co/EG6mffU86k', 'https://t.co/UYAwz...",Could acupuncture cure erectile dysfunction?\n...,,Could acupuncture cure erectile dysfunction?\n...


In [89]:
contractions = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [90]:
# Steps for text cleaning 

df = df.astype(str)

from spellchecker import SpellChecker

def text_cleaning(x):
    norm_text = x[0].upper() + x[1:] # Step I. Normalizing text
    for word in norm_text.split(): # Step II. Handling the contraction text
        if word.lower() in contractions:
            norm_text = norm_text.replace(word, contractions[word.lower()])
    rem_unicode = re.sub(r"[^-a-zA-Z0-9 /,:]+", "", norm_text) #Step III. Removing Unicode Characters
#     rem_unicode = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", norm_text) # Step III. Removing unicode characters
    return rem_unicode

spell = SpellChecker()

df['tweet_formatted'] = df['tweet_formatted'].apply(text_cleaning)
df['tweet_formatted'] = df['tweet_formatted'].apply(lambda x: spell.correction(x))

In [91]:
df.to_csv('Tweets_formatted_08012022.csv')

In [13]:
import pandas as pd
import re
import numpy as np

In [32]:
tweet=pd.read_excel(r"C:\Users\Revathi P\Documents\Graduate Research - Twitter Analysis\Datasets\Latest\data2207_094933.xlsx")

In [33]:
tweet.head(2)

,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,reply_settings,conversation_id,referenced_tweets_type,referenced_tweets_id,in_reply_to_user_id
0,2.300559e+09,2017-01-29 23:59:55+00:00,,8.258560e+17,en,0.0,0.0,0.0,0.0,Twitter for iPhone,I have erectile dysfunction â˜¹ï¸_x008f_,everyone,8.258560e+17,NaN,NaN,NaN
1,1.670847e+08,2017-01-29 23:20:09+00:00,,8.258460e+17,en,0.0,0.0,0.0,34.0,Twitter for iPhone,RT @jokesuk: Anyone had to deal with erectile ...,everyone,8.258460e+17,"[{'type': 'retweeted', 'id': '8212702208410869...","[{'type': 'retweeted', 'id': '8212702208410869...",NaN


In [34]:
tweet=tweet[['tweet']]

In [35]:
tweet.iloc[0]

tweet    I have erectile dysfunction â˜¹ï¸_x008f_
Name: 0, dtype: object

In [37]:
tweet=tweet[tweet['tweet'].isnull()==False]

In [38]:
tweet['Tweet_exc_link']=tweet['tweet'].apply(lambda x: re.sub(r"http\S+", "",x.lower()))

In [39]:
tweet.to_csv(r'C:\Users\Revathi P\Documents\Graduate Research - Twitter Analysis\Datasets\Archive\chk.csv')